This notebook presents the various elements of the DTW component of the predictive system. The actual module is stored in the `libdtw.py` file.

## Preliminaries: Data Loading
The `libdtw.py` module contains 2 functions:
- `load_data(n_to_keep=50, data_path = "data/ope3_26.pickle")`
- `assign_ref(data)`

and one class:
- `Dtw(json_obj = False)`

We first illustrate the usage of the two funcitons.

`load_data(n_to_keep=50, data_path = "data/ope3_26.pickle")` load the .pickle in the `data_path` position. This has to be a dictionary where the keys are the batch IDs and the values are lists of dictionaries representing the PVs (with keys `name, start, end, values`). 

The function first identifies the median (with respect to the duration parameter) batch to be used as reference. Then, it selects the first `n_to_keep` batches closer to the reference one in terms of duration. The output dictionary has the `reference` key explicitly declared.

In [ ]:
def load_data(n_to_keep=50, data_path = "data/ope3_26.pickle"):
    """
    Load data of operation 3.26, only the n_to_keep batches with duration closer to the median one
    are selected
    """
    with open(data_path, "rb") as infile:
        data = pickle.load(infile)

    operation_length = list()
    pv_dataset = list()
    for _id, pvs in data.items():
        operation_length.append((len(pvs[0]['values']), _id))
        pv_list = list()
        for pv_dict in pvs:
            pv_list.append(pv_dict['name'])
        pv_dataset.append(pv_list)

    median_len = np.median([l for l, _id in operation_length])

    # Select the batches closer to the median bacth
    # center around the median
    centered = [(abs(l-median_len), _id) for l, _id in operation_length]
    selected = sorted(centered)[:n_to_keep]

    med_id = selected[0][1]

    all_ids = list(data.keys())
    for _id in all_ids:
        if _id not in [x[1] for x in selected]:
            _ = data.pop(_id)

    data['reference'] = med_id

    return data

`assign_ref(data)` computes the median batch (acording to the duration parameter) and sets it as `reference` of the data set. It is useful in case the data loaded with `load_data()` undergoes modification prior to its actual use

In [ ]:
def assign_ref(data):
    data = copy(data)
    operation_length = list()
    pv_dataset = list()
    for _id, pvs in data.items():
        operation_length.append((len(pvs[0]['values']), _id))
        pv_list = list()
        for pv_dict in pvs:
            pv_list.append(pv_dict['name'])
        pv_dataset.append(pv_list)

    median_len = np.median([l for l, _id in operation_length])

    # Select the ref_len=50 closest to the median bacthes
    # center around the median
    centered = [(abs(l-median_len), _id) for l, _id in operation_length]
    selected = sorted(centered)

    med_id = selected[0][1]  # 5153

    all_ids = list(data.keys())
    for _id in all_ids:
        if _id not in [x[1] for x in selected]:
            _ = data.pop(_id)

    data['reference'] = med_id

    return data

## Dtw class
The `Dtw(json_obj)` contains all the methods used to set up the DTW algorithm, optimizing the variables weights, computing the alignment. it contains the following methods (divided by topic here for clarity):

##### data handling
- `__init__()`
- `convert_data_from_json()`
- `add_query`
- `get_scaling_parameters`
- `remove_const_feats`
- `scale_pv`
- `convert_to_mvts`

##### dtw implementation
- `comp_dist_matrix`
- `comp_acc_dist_matrix`
- `comp_acc_element`
- `get_warping_path`
- `call_dtw`
- `dtw`
- `get_ref_prefix_length`
- `itakura`
- `extreme_itakura`
- `check_open_ended`

##### step pattern selection utilities
- `time_distortion`
- `avg_time_distortion`
- `avg_distance`
- `get_p_max`
- `get_global_p_max`

##### variables weights optimization
- `reset_weights`
- `compute_mld`
- `extract_single_feat
- `weight_optimization_single_batch`
- `weight_optimization_step`
- `optimize_weights`
- `get_weight_variables`

##### visualization
- `distance_cost_plot`
- `plot_weights`
- `plot_by_name`
- `do_warp`
- `plot_warped_curves`

##### misc
- `online_scale`
- `online_query`
- `generate_train_set`

We now examine each method.



### Data handling
`__init__(json_obj=False, random_weights = True, scaling='group')` initialize the `Dtw` class. 

- `json_obj` is the dictionary containing the data in the output format of `load_data()`. 
- `random_weights` if True, initialize the variable weights to randomly chosen values in the [0.1, 1] interval. 
- `scaling` is the scaling strategy for the PVs: `group` scales the PVs according to the values of the PVs in the reference batch, `single` scales the PVs as individual entities

The initialization consists in structuring the data inside the Dtw object, removing the constant features (in the reference batch) filtering the batches that does not contain all the PVs of the reference batch, and finally setting the variables weights.

In [ ]:
def __init__(self, json_obj=False, random_weights = True, scaling='group'):
    """
    Initialization of the class.
    json_obj: contains the data in the usual format
    """
    if not json_obj:
        pass
    else:
        self.convert_data_from_json(deepcopy(json_obj))
        #self.scale_params = self.get_scaling_parameters()
        self.remove_const_feats()
        self.reset_weights(random=random_weights)
        self.scaling = scaling

---
`convert_data_from_json(json_obj)` takes as input the data object specified before and separates it into reference and query batches. It also initialize the internal structure of the `Dtw` object. This includes the structure to collect data useful in subsequent operations and avoid repeating calculations.

In [ ]:
def convert_data_from_json(self, json_obj):
        """
        Returns a dictionary containing all the data, organized as:
        ref_id: the ID of the reference batch
        reference: reference batch in the usual format (list of dictionaries)
        queries: list of dictionaries in which the keys are the query batch's ID and the values are
        the actual batches (list of dictionaries)
        num_queries: number of query batches in the data set
        """
        ref_id = json_obj["reference"]
        reference = json_obj[ref_id]
        queries = {key: batch for key, batch in json_obj.items() if key !=
                   "reference" and key != ref_id}

        self.data = {"ref_id": ref_id,
                     "reference": reference,
                     "queries": queries,
                     "num_queries": len(queries),
                     "warpings": dict(),
                     "distances": dict(),
                     'warp_dist': dict(),
                     "queriesID": list(queries.keys()),
                     "time_distortion": defaultdict(dict),
                     "distance_distortion": defaultdict(dict),
                     'warpings_per_step_pattern': defaultdict(dict),
                     'feat_weights': 1.0}

        self.data_open_ended = {"ref_id": ref_id,
                                "reference": reference,
                                "queries": defaultdict(list),
                                'warp_dist': dict()}
        scale_params = dict()

        for pv_dict in self.data['reference']:
            pv_name = pv_dict['name']
            pv_min = min(pv_dict['values'])
            pv_max = max(pv_dict['values'])
            scale_params[pv_name] = (pv_min, pv_max)

        self.scale_params = scale_params

---
`add_query(batch_dict)` adds a batch (in the forms of a dict(batch_id: list_of_PVs)) to the `Dtw` object, updating the relevant structures and filtering the PVs. If the batch does not contain all the necessary PVs, it is not added.

In [ ]:
def add_query(self, batch_dict):
    _id, pvs = list(batch_dict.items())[0]
    self.data['queries'][_id] = pvs
    self.data['num_queries'] += 1
    self.data['queriesID'].append(_id)
    self.remove_const_feats()

---
`get_scaling_parameters()` probably can be omitted, TO CHECK

In [ ]:
def get_scaling_parameters(self):
        """
        Computes the parameters necessary for scaling the features as a 'group'.
        This means considering the mean range of a variable across al the data set.
        This seems creating problems, since the distributions for the minimum and the
        maximum are too spread out. This method is here just in case of future use and to help
        removing non-informative (constant) features.
        avg_range = [avg_min, avg_max]
        """
        scale_params = dict()

        for pv_dict in self.data['reference']:
            pv_name = pv_dict['name']
            pv_min = min(pv_dict['values'])
            pv_max = max(pv_dict['values'])

            scale_params[pv_name] = [[pv_min], [pv_max]]

        for _id, batch in self.data['queries'].items():
            for pv_dict in batch:
                pv_name = pv_dict['name']
                pv_min = min(pv_dict['values'])
                pv_max = max(pv_dict['values'])

                scale_params[pv_name][0].append(pv_min)
                scale_params[pv_name][1].append(pv_max)

        pv_names = scale_params.keys()
        for pv_name in pv_names:
            scale_params[pv_name] = np.median(scale_params[pv_name], axis=1)

        return scale_params

---
`remove_const_feats()` removes the features that are constant in the reference batch from all the batches, reference and queries. If a query batch does not contain all the PVs of the filtered reference one, it is removed from the `Dtw` object. 

If a Pv is constant in the reference batch, this does not mean that it is constant also in any of the queries, but since all the DTW alignment are performed with respect to one single reference, this PVs would add no information.

In [ ]:
def remove_const_feats(self):
        """
        Removes non-informative features (features with low variability)
        """
        const_feats = list()
        for pv_name, avg_range in self.scale_params.items():
            if abs(avg_range[0]-avg_range[1]) < 1e-6:
                const_feats.append(pv_name)
        #const_feats.append('ba_TCzWpXo')
        #const_feats.append('ba_TCfg3Yxn')
        #const_feats.append('ba_FQYXdr6Q0')


        initial_queries = list(self.data['queries'].keys())
        print('Number of queries before filtering: %d'%len(initial_queries))

        self.data['reference'] = list(filter(lambda x: x['name'] not in const_feats, self.data['reference']))
        pv_names = [pv['name'] for pv in self.data['reference']]
        for _id in initial_queries:
            self.data['queries'][_id] = list(filter(lambda x: x['name']  in pv_names, self.data['queries'][_id]))
            if len(self.data['queries'][_id]) != len(self.data['reference']):
                _ = self.data['queries'].pop(_id)
        print('Number of queries after filtering: %d'%len(self.data['queries']))

        self.data['num_queries'] = len(self.data['queries'])
        self.data['queriesID'] = list(self.data['queries'].keys())
        self.pv_names = pv_names

---
`scale_pv(pv_name, pv_values, mode="single")` takes as parameters the `pv_name` in order to get the right scaling parameters from the structure where they are stored, the `pv_values` to scale in the form of a list, and the mode of scaling:

-`single` scales PV independently from the others: thus computing min and max of the PV and scaling to the [0, 1] interval. This is apt to off-line scenarios
- `group` scales the PVs with respect to values of the same PV in the reference batch. This way, the online and offline scenarios are treated equally from this point of view

The scaling formula adopted is:
$$X_{scaled} = \frac{X_{original} - min}{max - min}$$
where min and max have different meaning for `single` and `group` scaling. If a PV is constant in any one of the queries, it is scaled by default to the constant 0.5 value.

In [ ]:
def scale_pv(self, pv_name, pv_values, mode="single"):
        """
        Scales features in two possible ways:
            'single': the feature is scaled according to the values it assumes in the current batch
            'group': the feature is scaled according to its average range across the whole data set
        """
        if mode == "single":
            pv_min = min(pv_values)
            pv_max = max(pv_values)
            if abs(pv_max-pv_min) > 1e-6:
                scaled_pv_values = (np.array(pv_values)-pv_min)/(pv_max-pv_min)
            else:
                scaled_pv_values = .5 * np.ones(len(pv_values))
        elif mode == "group":
            pv_min, pv_max = self.scale_params[pv_name]
            scaled_pv_values = (np.array(pv_values)-pv_min)/(pv_max-pv_min)
        return scaled_pv_values

---
`convert_to_mvts(batch)` takes as input a batch in the usual format (list of PV dictionaries) and turns it to a numpy array for faster computation. The values of each PV (columns of the numpy array) are scaled according to the method specified at initialization.

In [ ]:
def convert_to_mvts(self, batch):     # mvts = Multi Variate Time Series
        """
        Takes one batch in the usual form (list of one dictionary per PV) and transforms
        it to a numpy array to perform calculations faster
        """
        k = len(batch[0]['values'])  # Length of a batch (number of data points per single PV)
        num_feat = len(batch)  # Number of PVs

        mvts = np.zeros((k, num_feat))

        for (i, pv_dict) in zip(np.arange(num_feat), batch):
            mvts[:, i] = self.scale_pv(pv_dict['name'], pv_dict['values'], "group")

        return mvts

### DTW implementation
`comp_dist_matrix(reference_ts, query_ts, n_jobs=1)` takes as input the reference and a query batch in the mvts form (returned by `convert_to_mvts(batch)`) and computes the local distance matrix using the eucliden distance measure. The `pairwise_distances` function takes the weight for the features from those stored in the `Dtw` object. `n_jobs` is used for parallelization (not always faster that single core computation)


In [ ]:
def comp_dist_matrix(self, reference_ts, query_ts, n_jobs=1):
        """
        Computes the distance matrix with ref_len (length of the reference) number of rows and
        query_len (length of the query) number of columns (OK with convention on indices in dtw)
        with dist_measure as local distance measure

        reference_ts: mvts representation of reference batch
        query_ts: mvts representation of query batch

        n_jobs: number of jobs for pairwise_distances function. It could cause problems on windows
        """
        _, d_1 = reference_ts.shape
        _, d_2 = query_ts.shape

        if d_1 != d_2:
            print("Number of features not coherent between reference ({0}) and query ({1})"
                  .format(d_1, d_2))
            return None

        distance_matrix = pairwise_distances(
            X=reference_ts, Y=query_ts, metric=euclidean, n_jobs=n_jobs, w=self.data['feat_weights']
            )

        return distance_matrix

---
`comp_acc_dist_matrix(distance_matrix, step_pattern='symmetricP05', open_ended=False)` computes the accumulated distance matrix starting from the local distance matrix according to the specified step pattern. Probably the `open_ended` parameter is superfluous.

In [ ]:
def comp_acc_dist_matrix(self, distance_matrix, step_pattern='symmetricP05', open_ended=False):
        """
        Computes the accumulated distance matrix starting from the distance_matrix according to the
        step_pattern indicated
        distance_matrix: cross distance matrix
        step_pattern: string indicating the step pattern to be used. Can be symmetric1/2,
        symmetricP05 or symmetricPX, with X any positive integer
        """
        ref_len, query_len = distance_matrix.shape
        acc_dist_matrix = np.empty((ref_len, query_len))
        if not open_ended:
            for i in np.arange(ref_len):
                for j in np.arange(query_len):
                    acc_dist_matrix[i, j] = self.comp_acc_element(
                        i, j, acc_dist_matrix, distance_matrix, step_pattern)\
                        if self.itakura(i, j, ref_len, query_len, step_pattern) else np.inf

        else:
            for i in np.arange(ref_len):
                for j in np.arange(query_len):
                    acc_dist_matrix[i, j] = self.comp_acc_element(
                        i, j, acc_dist_matrix, distance_matrix, step_pattern)
        return acc_dist_matrix


---
`comp_acc_element(i, j, acc_dist_matrix, distance_matrix, step_pattern)` computes the element (i,j) of the accumulated distance matrix according to the specified step pattern. The possible step patterns are `symmetric1`, `symmetric2`, `symmetricP05` and `symmetricPN` for N positive integer. 

In [ ]:
def comp_acc_element(self, i, j, acc_dist_matrix, distance_matrix, step_pattern):
        """
        Computes the value of a cell of the accumulated distance matrix
        i: row (reference) index
        j: column (query) index
        acc_dist_matrix: current accumulated distance matrix
        distance_matrix: cross distance matrix
        step_pattern: step pattern to be used for calculations
        """
        if (i == 0 and j == 0):
            return distance_matrix[0, 0]

        if step_pattern == "symmetricP05":

            p_1 = acc_dist_matrix[i-1, j-3] + 2 * distance_matrix[i, j-2] + distance_matrix[i, j-1]\
                + distance_matrix[i, j] if (i-1 >= 0 and j-3 >= 0) else np.inf
            p_2 = acc_dist_matrix[i-1, j-2] + 2 * distance_matrix[i, j-1] + \
                distance_matrix[i, j] if (i-1 >= 0 and j-2 >= 0) else np.inf
            p_3 = acc_dist_matrix[i-1, j-1] + 2 * \
                distance_matrix[i, j] if (i-1 >= 0 and j-1 >= 0) else np.inf
            p_4 = acc_dist_matrix[i-2, j-1] + 2 * distance_matrix[i-1, j] + \
                distance_matrix[i, j] if (i-2 >= 0 and j-1 >= 0) else np.inf
            p_5 = acc_dist_matrix[i-3, j-1] + 2 * distance_matrix[i-2, j] + distance_matrix[i-1, j]\
                + distance_matrix[i, j] if (i-3 >= 0 and j-1 >= 0) else np.inf

            return min(p_1, p_2, p_3, p_4, p_5)  

        if step_pattern == "symmetric1":
            p_1 = acc_dist_matrix[i, j-1] + distance_matrix[i, j] if (j-1 >= 0) else np.inf
            p_2 = acc_dist_matrix[i-1, j-1] + distance_matrix[i, j]\
                if (i-1 >= 0 and j-1 >= 0) else np.inf
            p_3 = acc_dist_matrix[i-1, j] + distance_matrix[i, j] if (i-1 >= 0) else np.inf

            return min(p_1, p_2, p_3)

        if step_pattern == "symmetric2":
            p_1 = acc_dist_matrix[i, j-1] + distance_matrix[i, j] if (j-1 >= 0) else np.inf
            p_2 = acc_dist_matrix[i-1, j-1] + 2 * \
                distance_matrix[i, j] if (i-1 >= 0 and j-1 >= 0) else np.inf
            p_3 = acc_dist_matrix[i-1, j] + distance_matrix[i, j] if (i-1 >= 0) else np.inf

            return min(p_1, p_2, p_3)  

        patt = re.compile("symmetricP[1-9]+\d*")
        if patt.match(step_pattern):
            p = int(step_pattern[10:])
            p_1 = acc_dist_matrix[i-p, j-(p+1)] + 2*sum([distance_matrix[i-p, j-(p+1)]\
                                             for p in np.arange(0, p)]) + distance_matrix[i, j]\
                                                        if (i-p >= 0 and j-(p+1) >= 0) else np.inf
            p_2 = acc_dist_matrix[i-1, j-1] + \
                2 * distance_matrix[i, j] if (i-1 >= 0 and j-1 >= 0) else np.inf
            p_3 = acc_dist_matrix[i-(p+1), j-p] + 2*sum([distance_matrix[i-(p+1), j-p]\
                                             for p in np.arange(0, p)]) + distance_matrix[i, j] \
                                                                    if (i-(p+1) >= 0 and j-p >= 0) \
                                                                                        else np.inf

            return min(p_1, p_2, p_3)

---
`get_warping_path(acc_dist_matrix, step_pattern, ref_len, query_len)` computes the warping path on the accumulated distance matrix, coherent with the given step pattern. `ref_len` and `query_len` specify the last point of the warping path, in order to make the computation possible in the open-ended version too.

In [ ]:
def get_warping_path(self, acc_dist_matrix, step_pattern, ref_len, query_len):
        """
        Computes the warping path on the acc_dist_matrix induced by step_pattern starting from
        the (ref_len,query_len) point (this in order to use the method in both open_ended and global
        alignment)
        Return the warping path (list of tuples) in ascending order
        """
        # ref_len, query_len = acc_dist_matrix.shape
        warping_path = list()

        if step_pattern == "symmetric1" or step_pattern == "symmetric2":
            i = ref_len-1
            j = query_len-1
            while i != 0 or j != 0:
                warping_path.append((i, j))
                candidates = list()
                if i > 0:
                    candidates.append((acc_dist_matrix[i-1, j], (i-1, j)))
                if j > 0:
                    candidates.append((acc_dist_matrix[i, j-1], (i, j-1)))
                if len(candidates) == 2:
                    candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))

                next_step = min(candidates)[1]
                i, j = next_step
            warping_path.append((0, 0))

            return warping_path[::-1]

        elif step_pattern == "symmetricP05":
            # maxWarp = 2
            # minDiag = 1
            i = ref_len-1
            j = query_len-1

            if np.isnan(acc_dist_matrix[i, j]):
                print("Invalid value for P, \
                      a global alignment is not possible with this local constraint")
                return
            h_step = 0  # horizontal step
            v_step = 0  # vertical step
            d_step = 0  # diagonal step

            while i != 0 or j != 0:
                warping_path.append((i, j))
                candidates = list()

                if h_step > 0:
                    if h_step == 1:
                        if j > 0:
                            candidates.append((acc_dist_matrix[i, j-1], (i, j-1)))
                        if j > 0 and i > 0:
                            candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))
                    elif h_step == 2:
                        if j > 0 and i > 0:
                            candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))

                elif v_step > 0:
                    if v_step == 1:
                        if i > 0:
                            candidates.append((acc_dist_matrix[i-1, j], (i-1, j)))
                        if j > 0 and i > 0:
                            candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))
                    elif v_step == 2:
                        if j > 0 and i > 0:
                            candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))

                else:
                    if j > 0:
                        candidates.append((acc_dist_matrix[i, j-1], (i, j-1)))
                    if i > 0:
                        candidates.append((acc_dist_matrix[i-1, j], (i-1, j)))
                    if j > 0 and i > 0:
                        candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))

                next_step = min(candidates)[1]
                v = next_step[0] < i
                h = next_step[1] < j
                d = v and h

                if d:
                    v_step = 0
                    h_step = 0
                elif v:
                    v_step += 1
                elif h:
                    h_step += 1

                i, j = next_step

            warping_path.append((0, 0))

            return warping_path[::-1]

        else:
            patt = re.compile("symmetricP[1-9]+\d*")
            if patt.match(step_pattern):

                min_diag_steps = int(step_pattern[10:])

                warp_step = 0
                d_step = 0
                i = ref_len-1
                j = query_len-1

                if np.isinf(acc_dist_matrix[i, j]):
                    print("Invalid value for P, \
                          a global alignment is not possible with this local constraint")
                    return

                while i != 0 and j != 0:
                    warping_path.append((i, j))
                    candidates = list()
                    if warp_step > 0:
                        candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))
                    else:
                        if j > 0:
                            candidates.append((acc_dist_matrix[i, j-1], (i, j-1)))
                        if i > 0:
                            candidates.append((acc_dist_matrix[i-1, j], (i-1, j)))
                        if len(candidates) == 2:
                            candidates.append((acc_dist_matrix[i-1, j-1], (i-1, j-1)))

                    next_step = min(candidates)[1]
                    v = next_step[0] < i
                    h = next_step[1] < j
                    d = v and h

                    if d:
                        d_step += 1
                        if d_step == min_diag_steps:
                            d_step = 0
                            warp_step = 0
                        elif d_step < min_diag_steps and warp_step > 0:
                            pass
                        elif d_step < min_diag_steps and warp_step == 0:
                            d_step = 0
                    else:
                        warp_step += 1

                    i, j = next_step

                warping_path.append((0, 0))

                return warping_path[::-1]

            else:
                print("Invalid step-pattern")


---
`call_dtw(query_id, step_pattern="symmetricP05", n_jobs=1, open_ended=False, get_results=False, length = 0, all_sub_seq=False)` calls the `dtw` method on the pair (reference, query) specified by `query_id`. `get_results` specifies if the method should return DTW distance, warping path and accumulated distance matrix as output or if it should just store the information inside the `Dtw` object. `length` refers to the query length to consider in case of open-ended DTW, and `all_sub_seq` computes the open-ended DTW for all the sub-sequences of the query series.

In [ ]:
def call_dtw(self, query_id, step_pattern="symmetricP05",
                 n_jobs=1, open_ended=False, get_results=False, length = 0, all_sub_seq=False):
        """
        Calls the dtw method on the data stored in the .data attribute (needs only the query_id in \
        addition to standard parameters)
        get_results if True returns the distance and the warping calculated; if False, \
        only the .data attribute is updated
        """
        if not open_ended:
            if step_pattern in self.data['warpings_per_step_pattern']:
                if query_id in self.data['warpings_per_step_pattern'][step_pattern]:
                    return

            reference_ts = self.convert_to_mvts(self.data['reference'])
            query_ts = self.convert_to_mvts(self.data['queries'][query_id])

            result = self.dtw(reference_ts, query_ts, step_pattern, n_jobs, open_ended)

            self.data["warpings"][query_id] = result["warping"]
            self.data["distances"][query_id] = result["DTW_distance"]
            self.data['warp_dist'][query_id]=list()
            for (i,j) in result["warping"]:
                self.data['warp_dist'][query_id].append((i, j, result['acc_matrix'][i, j]/(i+j+2)))

            self.data['time_distortion'][step_pattern][query_id] = \
                self.time_distortion(result['warping'])
            self.data['distance_distortion'][step_pattern][query_id] = result["DTW_distance"]
            self.data['warpings_per_step_pattern'][step_pattern][query_id] = result['warping']

            if get_results:
                return result

        if open_ended:
            # ADD ALL SUB SEQUENCE OPTION#############################
            if all_sub_seq:
                reference_ts = self.convert_to_mvts(self.data['reference'])
                query_ts = self.convert_to_mvts(self.data['queries'][query_id])

                result = self.dtw(reference_ts, query_ts, step_pattern, n_jobs, open_ended=False)

                acc_dist_matrix = result['acc_matrix']
                N, M = acc_dist_matrix.shape
                self.data_open_ended['warp_dist'][query_id] = list()
                for j in np.arange(M):
                    candidate = acc_dist_matrix[:, j]
                    candidate /= np.arange(2, N+2) + j
                    i_min, dtw_dist = np.argmin(candidate), min(candidate)

                    self.data_open_ended['warp_dist'][query_id].append((i_min, j, dtw_dist))
                return

            if not length:
                print("Length cannot be 0")
                return

            if not self.check_open_ended(query_id, length, step_pattern):

                query_ts = self.convert_to_mvts(self.online_query(query_id, length))
                reference_ts = self.convert_to_mvts(self.data['reference'])

                result = self.dtw(reference_ts, query_ts, step_pattern, n_jobs, open_ended)

                data_point = {'length': length,
                              'DTW_distance': result['DTW_distance'],
                              'warping':result['warping'],
                              'step_pattern': step_pattern}
                self.data_open_ended['queries'][query_id].append(data_point)

            if get_results:
                return list(filter(lambda x: x['step_pattern']==step_pattern and x['length']==length, self.data_open_ended['queries'][query_id]))[0]

---
`dtw(reference_ts, query_ts, step_pattern="symmetricP05", n_jobs=1, open_ended=False)` is the method that actually performs the DTE computation. It accepts series in the mvts format, and in case of standard (not open-ended) version, checks that the step pattern allows for a global alignment. The output is a dictionary containing the DTW distance, the warping path and the accumulated distance matrix.

In [ ]:
def dtw(self, reference_ts, query_ts, step_pattern="symmetricP05",
            n_jobs=1, open_ended=False):
        """
        Compute alignment betwwen reference_ts and query_ts (already in mvts form).
        Separate from call_dtw() for testing purposes
        """
        # Check for coherence of local constraint and global alignment
        # (in case a PX local constraint is used)
        if not open_ended:
            patt = re.compile("symmetricP[1-9]+\d*")
            if patt.match(step_pattern):
                p = int(step_pattern[step_pattern.index("P")+1:])
                ref_len, query_len = len(reference_ts), len(query_ts)
                p_max = np.floor(min(ref_len, query_len)/np.abs(ref_len-query_len)) \
                    if np.abs(ref_len-query_len) > 0 else np.inf
                if p > p_max:
                    print("Invalid value for P, \
                                  a global alignment is not possible with this local constraint")
                    return
            else:
                pass

        distance_matrix = self.comp_dist_matrix(reference_ts, query_ts, n_jobs)

        acc_dist_matrix = self.comp_acc_dist_matrix(distance_matrix, step_pattern, open_ended)

        ref_len, query_len = acc_dist_matrix.shape
        # In case of open-ended version
        # correctly identifies the starting point on the reference batch for warping
        if open_ended:
            ref_len = self.get_ref_prefix_length(acc_dist_matrix)

        warping = self.get_warping_path(acc_dist_matrix, step_pattern, ref_len, query_len)

        dtw_dist = acc_dist_matrix[ref_len-1, query_len-1] / (ref_len+query_len) if step_pattern != "symmetric1" \
                                                                    else acc_dist_matrix[ref_len-1, query_len-1]

        return {"warping": warping,
                "DTW_distance": dtw_dist,
                'acc_matrix': acc_dist_matrix}

---
`get_ref_prefix_length(acc_dist_matrix)` This method is called in case of open-ended DTW: it computes the length of the reference prefix to which the query is mapped.

In [ ]:
def get_ref_prefix_length(self, acc_dist_matrix):
        """
        Computes the length of the reference prefix in case of open-ended alignment
        """
        N, M = acc_dist_matrix.shape
        last_column = acc_dist_matrix[:, -1]/np.arange(1, N+1)

        ref_prefix_len = np.argmin(last_column) + 1
        return ref_prefix_len

---
`itakura(i, j, ref_len, query_len, step_pattern)` checks if the accumulated distance matrix cell in position (i,j) is or is not inside the Itakura parallelogram that the step pattern (of the `symmetricPN` class) imposes to the accumulated distance matrix.

In [ ]:
def itakura(self, i, j, ref_len, query_len, step_pattern):
        """
        Induced Itakura global constraint for GLOBAL ALIGNMENT
        """
        patt = re.compile("symmetricP[1-9]+\d*")
        if step_pattern == "symmetricP05":
            p = 1/2
        elif patt.match(step_pattern):
            p = int(step_pattern[step_pattern.index('P')+1:])
        else:
            return True

        in_domain = (i >= np.floor(j*p/(p+1))) and \
                    (i <= np.ceil(j*(p+1)/p)) and \
                    (i <= np.ceil(ref_len+(j-query_len)*(p/(p+1)))) and \
                    (i >= np.floor(ref_len+(j-query_len)*((p+1)/p)))
        return in_domain

---
`extreme_itakura(i, j, ref_len, query_len, step_pattern)` alternative implementation of the itakura method, could speed up the computation in some settings

In [ ]:
def extreme_itakura(self, i, j, ref_len, query_len, step_pattern):
        """
        Alternative implementation of itakura method
        """
        case = 0
        patt = re.compile("symmetricP[1-9]+\d*")
        if step_pattern == "symmetricP05":
            p = 1/2
        elif patt.match(step_pattern):
            p = int(step_pattern[step_pattern.index('P')+1:])
        else:
            return (case, True)

        if (i < np.floor(j*p/(p+1))) or (i < np.floor(ref_len+(j-query_len)*((p+1)/p))):
            case = 1
            return (case, False)

        in_domain = (i >= np.floor(j*p/(p+1))) and \
                    (i <= np.ceil(j*(p+1)/p)) and \
                    (i <= np.ceil(ref_len+(j-query_len)*(p/(p+1)))) and \
                    (i >= np.floor(ref_len+(j-query_len)*((p+1)/p)))

        return (case, in_domain)

---
`check_open_ended(query_id, length, step_pattern)` check if the open-ended DTW has already been computed, in order to avoid needless computation

In [ ]:
def check_open_ended(self, query_id, length, step_pattern):
        check_id = query_id in self.data_open_ended['queries']
        if check_id:
            check = bool(list(filter(
                lambda x: x['step_pattern'] == step_pattern and x['length'] == length, self.data_open_ended['queries'][query_id])))
            return check
        else:
            return False

### Step pattern selection utilities

`time_distortion(warping_path)` computes the time distortion of an alignment, counting the number of warping steps (non diagonal steps) the warping path contains.

In [ ]:
def time_distortion(self, warping_path):
        """
        Computes the time distortion caused by warping_path
        """
        T = len(warping_path)
        f_q = [w[1] for w in warping_path]
        f_r = [w[0] for w in warping_path]

        t_d = [(f_r[t+1] - f_r[t])*(f_q[t+1] - f_q[t]) == 0 for t in np.arange(T-1)]

        return sum(t_d)

---
`avg_time_distortion(step_pattern)` computes the average time distortion, relative to a certain step pattern, for the whole data set of queries contained in the `Dtw` object.

In [ ]:
def avg_time_distortion(self, step_pattern):
        """
        Computes the average time distortion relative to a certain step pattern
        """
        if len(self.data['time_distortion'][step_pattern]) != self.data['num_queries']:
            print('Not every query aligned, align the remaining queries')
            return
        else:
            I = self.data['num_queries']
            avg_td = sum(self.data['time_distortion'][step_pattern].values())/I

            return avg_td

---
`avg_distance(step_pattern)` computes the average DTW distance, relative to a certain step pattern, for the whole data set of queries contained in the `Dtw` object.

In [ ]:
def avg_distance(self, step_pattern):
        """
        Computes average
        """
        if len(self.data['distance_distortion'][step_pattern]) != self.data['num_queries']:
            print('Not every query aligned, align the remaining queries')
            return
        else:
            I = self.data['num_queries']
            avg_dist = sum(self.data['distance_distortion'][step_pattern].values())/I

            return avg_dist

---
`get_p_max(query_id)` computes the maximum value of the parameter P (for `symmetricPN` step patterns) that allows for a global alignment between the reference and the query with ID `query_id`.

In [ ]:
def get_p_max(self, query_id):
        """
        Computes the maximum value of P for the selected query batch
        """
        k_q = len(self.data['queries'][query_id][0]['values'])
        k_r = len(self.data['reference'][0]['values'])
        p_max = np.floor(min(k_q, k_r)/abs(k_q - k_r)) if abs(k_q - k_r) > 0 else k_r
        return p_max

---
`get_global_p_max():` computes the maximum value of the parameter P (for `symmetricPN` step patterns) that allows for a global alignment for every query in the `Dtw` object. This correspond to the minimum of the set of P-max for all the queries.

In [ ]:
def get_global_p_max(self):
        """
        Computes the maximum value of P for the data set under consideration
        """
        p_maxs = [self.get_p_max(query_id) for query_id in self.data['queriesID']]
        return int(min(p_maxs))

##### variables weights optimization
- `reset_weights`
- `compute_mld`
- `extract_single_feat
- `weight_optimization_single_batch`
- `weight_optimization_step`
- `optimize_weights`
- `get_weight_variables`


In [ ]:
def reset_weights(self, random=False):
        """
        Reset the variables' weights to 1 or to random [0,1] numbers
        """
        n_feat = len(self.data['reference'])
        if not random:
            weights = np.ones(n_feat)
        else:
            weights = np.abs(np.random.normal(loc=1.0, size=n_feat))
            weights = weights/sum(weights) * n_feat

        self.data['feat_weights'] = weights

##### visualization
- `distance_cost_plot`
- `plot_weights`
- `plot_by_name`
- `do_warp`
- `plot_warped_curves`

##### misc
- `online_scale`
- `online_query`
- `generate_train_set`